In [1]:
import torch
print(f"PyTorch: {torch.__version__}")
print(f"ROCm: {torch.version.roc}")
print(f"GPU доступна: {torch.cuda.is_available()}")
print(f"Название GPU: {torch.cuda.get_device_name(0)}")
print(f"Количество видеокарт: {torch.cuda.device_count()}")

ImportError: libamdhip64.so: cannot enable executable stack as shared object requires: Invalid argument

In [ ]:
import re

class TextPreprocessor:
    """
    Класс для очистки и предобработки текста.
    """

    __slots__ = (
        "url_pattern",
        "emoji_pattern",
        "html_pattern",
        "non_letter_pattern",
        "telegram_pattern",
        "spaces_pattern",
    )

    def __init__(self):
        """
        Инициализация класса TextPreprocessor.
        Предкомпилирует регулярные выражения для очистки текста.
        """
        self.url_pattern = re.compile(r"https?://\S+|www\.\S+")
        self.emoji_pattern = re.compile(
            "["
            "\U0001f600-\U0001f64f"  # эмоции
            "\U0001f300-\U0001f5ff"  # символы
            "\U0001f680-\U0001f6ff"  # транспорт
            "\U0001f700-\U0001f77f"  # алхимия
            "\U0001f780-\U0001f7ff"  # геометрические фигуры
            "\U0001f800-\U0001f8ff"  # дополнительные символы
            "\U0001f900-\U0001f9ff"  # дополнительные символы-2
            "\U0001fa00-\U0001fa6f"  # шахматы
            "\U0001fa70-\U0001faff"  # дополнительные символы-3
            "\U00002702-\U000027b0"  # Dingbats
            "\U000024c2-\U0001f251"  # Enclosed
            "]+",
            flags=re.UNICODE,
        )
        self.html_pattern = re.compile(r"&[a-z]+;")
        self.non_letter_pattern = re.compile(r"[^a-zа-я\s]")
        self.telegram_pattern = re.compile(r"@\w+|/\w+")
        self.spaces_pattern = re.compile(r"\s+")

    def __preprocess(
        self,
        text,
        lowercase=True,
        replace_yo=True,
        remove_urls=True,
        remove_emoji=True,
        remove_html=True,
        remove_punctuation=True,
        remove_telegram=True,
    ):
        """
        Приватный метод базовой предобработки текста.

        :param text: Исходный текст для предобработки.
        :param lowercase: Флаг для приведения текста к нижнему регистру.
        :param replace_yo: Флаг для замены буквы "ё" на "е".
        :param remove_urls: Флаг для удаления URL-ссылок.
        :param remove_emoji: Флаг для удаления эмодзи.
        :param remove_html: Флаг для удаления HTML-сущностей.
        :param remove_punctuation: Флаг для удаления пунктуации и не-буквенных символов.
        :param remove_telegram: Флаг для удаления телеграм-упоминаний и бот-команд.
        :return: Очищенный текст.
        """
        result = text

        if lowercase:
            result = result.lower()

        if replace_yo:
            result = result.replace("ё", "е")

        if remove_urls:
            result = self.url_pattern.sub(" ", result)

        if remove_emoji:
            result = self.emoji_pattern.sub(" ", result)

        if remove_html:
            result = self.html_pattern.sub(" ", result)

        if remove_punctuation:
            result = self.non_letter_pattern.sub(" ", result)

        if remove_telegram:
            result = self.telegram_pattern.sub(" ", result)

        return self.spaces_pattern.sub(" ", result).strip()

    def clear_text(self, text):
        """
        Полная очистка текста для семантического поиска.

        :param text: Исходный текст для очистки.
        :return: Очищенный текст.
        """
        return self.__preprocess(text)

    def clean_for_search(self, text):
        """
        Очистка текста для поисковых запросов.

        :param text: Исходный текст для очистки.
        :return: Очищенный текст.
        """
        return self.__preprocess(text, remove_punctuation=False, remove_telegram=False)

    def clean_for_embedding(self, text):
        """
        Очистка текста перед получением эмбеддингов.

        :param text: Исходный текст для очистки.
        :return: Очищенный текст.
        """
        return self.__preprocess(
            text, remove_emoji=True, remove_html=True, remove_punctuation=True
        )

    def clean_for_display(self, text):
        """
        Лёгкая очистка для отображения пользователю.

        :param text: Исходный текст для очистки.
        :return: Очищенный текст.
        """
        return self.__preprocess(
            text,
            lowercase=False,
            replace_yo=False,
            remove_urls=False,
            remove_emoji=False,
            remove_punctuation=False,
        )

    def get_text_stats(self, text):
        """
        Возвращает статистику по тексту: длина текста, количество слов,
        уникальные слова и другие полезные метрики.

        Args:
            text (str): Исходный текст для анализа

        Returns:
            dict: Словарь с различными статистическими метриками текста:
                - length: общая длина текста в символах
                - word_count: количество слов в тексте
                - unique_word_count: количество уникальных слов
                - word_frequencies: словарь частоты встречаемости слов
                - avg_word_length: средняя длина слова
                - short_words_count: количество слов длиной менее 4 символов
                - long_words_count: количество слов длиной более 7 символов
        """
        # Проверяем входные данные
        if not text or not isinstance(text, str):
            return {
                "length": 0,
                "word_count": 0,
                "unique_word_count": 0,
                "word_frequencies": {},
            }

        # Очищаем текст 
        cleaned_text = self.clear_text(text)

        # Разбиваем текст на слова
        words = cleaned_text.split()

        # Рассчитываем основные метрики
        word_count = len(words)
        unique_words = set(words)
        unique_word_count = len(unique_words)

        # Рассчитываем частоту слов
        word_frequencies = {}
        for word in words:
            word_frequencies[word] = word_frequencies.get(word, 0) + 1

        # Рассчитываем дополнительные метрики
        total_chars = sum(len(word) for word in words)
        avg_word_length = total_chars / word_count if word_count > 0 else 0
        short_words_count = sum(1 for word in words if len(word) < 4)
        long_words_count = sum(1 for word in words if len(word) > 7)

        # Собираем все метрики в словарь
        stats = {
            "length": len(cleaned_text),  # Длина текста в символах
            "word_count": word_count,  # Количество слов
            "unique_word_count": unique_word_count,  # Количество уникальных слов
            "word_frequencies": word_frequencies,  # Частота слов
            "avg_word_length": round(avg_word_length, 2),  # Средняя длина слова
            "short_words_count": short_words_count,  # Количество коротких слов (< 4 букв)
            "long_words_count": long_words_count,  # Количество длинных слов (> 7 букв)
        }

        return stats

In [ ]:
txt = 'Сегодня (26.07) премьера в 18.00 (мск) - долгожданный выпуск с [id1868874|Евгений Егоров]   https://youtu.be/a4HBdcgpjzI?si=kRUzKulyQ2GP_8fp'
text_rreprocessor = TextPreprocessor()
print(text_rreprocessor.clear_text(txt))

In [ ]:
print(text_rreprocessor.get_text_stats(txt))

---

In [ ]:
import re
from typing import List, Tuple, Dict, Optional

from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    Doc
)

class NatashaLinguisticAnalyzer:
    """
    Класс для лингвистического анализа русского текста с использованием Natasha.
    
    Выполняет токенизацию, лемматизацию, определение частей речи и
    сохраняет позиции символов каждого токена в оригинальном тексте.
    """
    
    __slots__ = ('segmenter', 'morph_vocab', 'emb', 'morph_tagger', 'text_preprocessor')
    
    def __init__(self):
        """
        Инициализирует компоненты Natasha для лингвистического анализа.
        
        Используемые компоненты:
        - Segmenter: Для сегментации текста на токены
        - MorphVocab: Для морфологического словаря и лемматизации
        - NewsEmbedding: Для векторных представлений
        - NewsMorphTagger: Для морфологической разметки (определения частей речи)
        - TextPreprocessor: Для предобработки текста
        """
        self.segmenter = Segmenter()
        self.morph_vocab = MorphVocab()
        self.emb = NewsEmbedding()
        self.morph_tagger = NewsMorphTagger(self.emb)

        # Инициализация экземпляра TextPreprocessor (композиция)
        self.text_preprocessor = TextPreprocessor()
    
    def clear_text_with_mapping(self, text: str) -> Tuple[str, List[int]]:
        """
        Очищает текст и возвращает маппинг индексов из очищенного текста в исходный.
        """
        # Получаем очищенный текст с помощью TextPreprocessor
        cleaned_text = self.text_preprocessor.clear_text(text)
        
        # Создаем маппинг индексов из очищенного текста в исходный
        mapping = []
        orig_index = 0
        
        for clean_char in cleaned_text:
            found = False
            # Пропускаем символы в исходном тексте до нахождения соответствия
            while orig_index < len(text) and not found:
                if text[orig_index].lower() == clean_char:
                    mapping.append(orig_index)
                    orig_index += 1
                    found = True
                else:
                    orig_index += 1
            
            # Если не найдено соответствие, используем последний известный индекс
            if not found:
                # Добавляем последний возможный индекс или -1, если текст пустой
                mapping.append(len(text) - 1 if text else -1)
        
        return cleaned_text, mapping

    
    def map_clean_to_original(self, clean_start: int, clean_end: int, mapping: List[int]) -> Tuple[int, int]:
        """
        Преобразует позицию в очищенном тексте в позицию в исходном тексте.
        """
        if not mapping:
            return -1, -1
        
        # Проверка границ
        if clean_start >= len(mapping):
            return -1, -1
        
        orig_start = mapping[clean_start]
        
        # Обработка случая, когда clean_end выходит за границы
        if clean_end - 1 >= len(mapping):
            # Используем последний элемент маппинга + некоторое смещение, 
            # чтобы захватить позиции после последнего маппированного символа
            orig_end = mapping[-1] + 2  # +2 для захвата большего контекста
        else:
            orig_end = mapping[clean_end - 1] + 1
        
        return orig_start, orig_end

    
    def analyze(self, text: str) -> Dict:
        """
        Выполняет лингвистический анализ текста с использованием Natasha.
        
        Args:
            text (str): Входной текст для анализа.
            
        Returns:
            Dict: Словарь с результатами анализа:
                - 'tokens': Список кортежей с информацией о токенах
                  (token, lemma, pos_tag, start_clean, end_clean, start_orig, end_orig)
                - 'original_text': Оригинальный текст (до очистки)
                - 'cleaned_text': Очищенный текст (после предобработки)
                - 'mapping': Маппинг индексов из очищенного текста в исходный
        """
        if not text or not isinstance(text, str):
            return {'tokens': [], 'original_text': '', 'cleaned_text': '', 'mapping': []}
        
        # Сохраняем оригинальный текст
        original_text = text
        
        # Базовая предобработка с TextPreprocessor и получение маппинга
        cleaned_text, mapping = self.clear_text_with_mapping(text)

        # Создаем объект Doc из Natasha
        doc = Doc(cleaned_text)
        
        # Сегментация текста на токены
        doc.segment(self.segmenter)
        
        # Морфологический анализ (определение частей речи)
        doc.tag_morph(self.morph_tagger)
        
        # Лемматизация токенов
        for token in doc.tokens:
            token.lemmatize(self.morph_vocab)
        
        # Создаем результирующий список с позициями как в очищенном, так и в исходном тексте
        tokens = []
        for token in doc.tokens:
            # Позиции в очищенном тексте
            start_clean = token.start
            end_clean = token.stop
            
            # Маппинг в позиции в исходном тексте
            start_orig, end_orig = self.map_clean_to_original(start_clean, end_clean, mapping)
            
            tokens.append((
                token.text,          # Оригинальный токен
                token.lemma,         # Лемматизированная форма
                token.pos,           # Часть речи
                start_clean,         # Начальный индекс в очищенном тексте
                end_clean,           # Конечный индекс в очищенном тексте
                start_orig,          # Начальный индекс в исходном тексте
                end_orig             # Конечный индекс в исходном тексте
            ))
        
        return {
            'tokens': tokens,
            'original_text': original_text,
            'cleaned_text': cleaned_text,
            'mapping': mapping
        }
    
    def analyze_query(self, text: str) -> Dict:
        """
        Анализирует поисковый запрос и возвращает информацию о токенах и текстах.
        
        Args:
            text (str): Исходный текст поискового запроса
            
        Returns:
            Dict: Словарь с результатами анализа (аналогично методу analyze)
        """
        return self.analyze(text)


In [ ]:

# Создаем анализатор
analyzer = NatashaLinguisticAnalyzer()

# Пример текста
txt = 'Сегодня (26.07) премьера в 18.00 (мск) - долгожданный выпуск с [id1868874|Евгений Егоров] https://youtu.be/a4HBdcgpjzI?si=kRUzKulyQ2GP_8fp'



# Выполняем анализ
result = analyzer.analyze(txt)

# Выводим результат
print("Результат анализа:")
print(result)



In [ ]:
xx = 'Твой лучший секс спрятан здесь 🔞  Делюсь каналом дипломированного сексолога. Крис взломала код классного секса, мастерски раскрепощает, знает миллион горячих техник и лучшие девайсы для взрослых 😻  Самые полезные посты здесь:   Отрезвляющий пост «Я все сама!»   Прокачай наездницу  Ролевая игра «VIP кинотеатр»   Техника оральных ласк 💣   Как занимается сeксом неудобная женщина   Кстати, Крис провела трехдневный безоплатный онлайн интенсив-«От бревна до Богини». Совместно с врачом и владельцем секс-шопа.   Скорее смотри записи, пока не удалила 🔞  https://t.me/sekretskris/1048   Здесь жарче, чем в аду 😈'

In [ ]:
# Выполняем анализ
result = analyzer.analyze(xx)

# Выводим результат
print("Результат анализа:")
print(result)

In [ ]:
import os
import torch
from pathlib import Path
from transformers import AutoModel, AutoTokenizer
from typing import List, Union

class SbertLargeNLU:
    def __init__(
        self,
        model_name: str = 'sberbank-ai/sbert_large_nlu_ru',
        model_dir: str = None,
        max_length: int = 512,
        enable_rocm: bool = True
    ):
        self.device = self._get_device(enable_rocm)
        self.model_name = model_name
        self.max_length = max_length
        self.root_dir = self._get_root_dir()
        self.model_dir = self._init_model_dir(model_dir)
        self.tokenizer, self.model = self._load_model()

    def _get_device(self, enable_rocm: bool) -> str:
        if enable_rocm and torch.cuda.is_available():
            os.environ['TORCH_ROCM_AOTRITON_ENABLE_EXPERIMENTAL'] = '1'
            return 'cuda'
        return 'cpu'

    def _get_root_dir(self) -> Path:
        try:
            current_file = Path(__file__).resolve()
            return current_file.parent.parent if "src" in current_file.parts else current_file.parent
        except NameError:
            return Path(os.getcwd()).resolve()

    def _init_model_dir(self, model_dir: str) -> Path:
        path = Path(model_dir) if model_dir else self.root_dir / "models/sbert"
        path.mkdir(parents=True, exist_ok=True)
        return path

    def _load_model(self):
        local_path = self.model_dir / self.model_name.replace("/", "__")
        try:
            if (local_path / "config.json").exists():
                tokenizer = AutoTokenizer.from_pretrained(local_path)
                model = AutoModel.from_pretrained(local_path)
            else:
                tokenizer = AutoTokenizer.from_pretrained(self.model_name)
                model = AutoModel.from_pretrained(self.model_name)
                model.save_pretrained(local_path, safe_serialization=True)
                tokenizer.save_pretrained(local_path)
            
            return tokenizer, model.to(self.device)
        except Exception as e:
            raise RuntimeError(f"Failed to load model: {str(e)}")

    def create_embeddings(self, texts: Union[str, List[str]]) -> torch.Tensor:
        if not texts:
            return torch.empty((0,))
            
        if isinstance(texts, str):
            texts = [texts]
            
        texts = [t.strip() for t in texts if isinstance(t, str) and t.strip()]
        if not texts:
            return torch.empty((0,))
        
        inputs = self.tokenizer(
            texts,
            padding=True,
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        ).to(self.device)
        
        with torch.no_grad():
            outputs = self.model(**inputs)
            
        return self._mean_pooling(outputs, inputs['attention_mask'])

    @staticmethod
    def _mean_pooling(model_output, attention_mask):
        if attention_mask.sum() == 0:
            return torch.zeros((1, model_output.last_hidden_state.size(-1)))
            
        token_embeddings = model_output.last_hidden_state
        input_mask_expanded = (
            attention_mask
            .unsqueeze(-1)
            .expand(token_embeddings.size())
            .float()
        )
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


In [ ]:
model = SbertLargeNLU()
print(model.device)
